In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resnet-notop
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/drowsy
/kaggle/input/studentengagement/Stude

# Experiment 1: EngageV1 Model

In [2]:
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_images_and_labels(dataset_path):
    images = []
    labels = []

    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                img = cv2.imread(image_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (256, 256)) 
                images.append(img)
                labels.append(label)

    return images, labels

def preprocess_data(images, labels):
    label_dict = {"engaged": 3, "frustrated": 1, "confused": 2, "drowsy": -3, "bored": -2, "Looking Away": -1}
    numerical_labels = [label_dict[label] for label in labels]

    images = np.array(images)
    numerical_labels = np.array(numerical_labels)

    images = images / 255.0

    categorical_labels = to_categorical(numerical_labels, num_classes=len(label_dict))

    return images, categorical_labels

engaged = "/kaggle/input/studentengagement/Student-engagement-dataset/Engaged"
not_engaged = "/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged"

images1, labels1 = load_images_and_labels(engaged)
images2, labels2 = load_images_and_labels(not_engaged)
images = images1 + images2
labels = labels1 + labels2

processed_images, processed_labels = preprocess_data(images, labels)

train_images, test_images, train_labels, test_labels = train_test_split(
    processed_images, processed_labels, test_size=0.2, random_state=42
)


2024-02-01 19:51:59.445022: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 19:51:59.445142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 19:51:59.604375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
np.unique(train_labels)

array([0., 1.], dtype=float32)

In [4]:
train_images.shape

(1696, 256, 256, 3)

In [5]:
test_labels[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

base_model = tf.keras.applications.ResNet50(weights=None, include_top=False, input_shape=(256, 256, 3))
base_model.load_weights('/kaggle/input/resnet-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop (1).h5')

model = models.Sequential()
model.add(base_model)

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(
    datagen.flow(train_images, train_labels, batch_size=8),
    epochs=100,
    validation_data=(test_images, test_labels)
)

In [20]:
model.evaluate(test_images, test_labels)[1]

14/14 [==============================] - 1s 70ms/step - loss: 0.1486 - accuracy: 0.9316


0.9316037893295288

In [21]:
model.save('my_model.h5')

In [10]:
test_labels[5]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [12]:
import cv2
import numpy as np

img = cv2.imread('/kaggle/input/studentengagement/Student-engagement-dataset/Not engaged/bored/0001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (256, 256))

# img = test_images[5]
img = img / 255.0
img = np.expand_dims(img, axis=0)

model.predict(img)

1/1 [==============================] - 2s 2s/step


array([[7.68123343e-10, 1.11273956e-04, 3.79758518e-07, 3.81908103e-06,
        9.99882936e-01, 1.57169438e-06]], dtype=float32)

In [ ]:
img[0]

In [ ]:
model.summary()

## Experiment 2: EngagePlus model

In [20]:
import numpy as np

X=np.load('/kaggle/input/face-images/preprocessed_data/X_train.npy')
y=np.load('/kaggle/input/face-images/preprocessed_data/y_train.npy')
y=np.where(y>2, 1, 0)

In [22]:
np.unique(y)

array([0, 1])

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

y=to_categorical(y, num_classes=2)

train_images, test_images, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
np.unique(train_labels)

array([0., 1.], dtype=float32)

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

base_model = tf.keras.applications.ResNet50(weights=None, include_top=False, input_shape=(256, 256, 3))
base_model.load_weights('/kaggle/input/resnet-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop (1).h5')

model = models.Sequential()
model.add(base_model)

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_8 (Dense)             (None, 64)                131136    
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 2)                 130       
                                                                 
Total params: 23718978 (90.48 MB)
Trainable params: 23665858 (90.28 MB)
Non-trainable params: 53120 (207.50 KB)
________

In [25]:
train_images = np.stack((train_images,)*3, axis=-1)
test_images = np.stack((test_images,)*3, axis=-1)

train_images.shape

(1665, 256, 256, 3)

In [ ]:
model.fit(
    datagen.flow(train_images, train_labels, batch_size=8),
    epochs=100,
    validation_data=(test_images, test_labels)
)

In [32]:
model.evaluate(test_images, test_labels)[1]

14/14 [==============================] - 1s 66ms/step - loss: 0.0264 - accuracy: 0.9976


0.9976019263267517

In [19]:
model.save('engagev2.h5')

In [34]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(test_images)

y_pred = np.argmax(y_pred, axis=1)

y_real = np.argmax(test_labels, axis=1)

print(classification_report(y_real, y_pred))

cm = confusion_matrix(y_real, y_pred)

print(cm)

14/14 [==============================] - 1s 68ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       215
           1       1.00      1.00      1.00       202

    accuracy                           1.00       417
   macro avg       1.00      1.00      1.00       417
weighted avg       1.00      1.00      1.00       417

[[214   1]
 [  0 202]]


In [35]:
model.save('engageultra.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
